In [3]:
import spacy
import os
from bs4 import BeautifulSoup
from lxml import etree
import numpy as np

nlp = spacy.load('en_core_web_sm', disable=['tagger','parser','lemmatizer','textcat', 'ner'])


def info(str_):
    print(f'{datetime.datetime.now()} [ INFO  ] {str_}')
def ok(str_):
    print(f'{datetime.datetime.now()} [ OK    ] {str_}')
def warning(str_):
    print(f'{datetime.datetime.now()} [WARNING] {str_}')

TS_data_path = '/home/ec2-user/SageMaker/data/Toronto_Star_Publication_with_query/'
GM_data_path = '/home/ec2-user/SageMaker/data/The_Globe_and_Mail_with_DP_filter_by_article_type/'
vectors_path = '/home/ec2-user/SageMaker/mariano/notebooks/03. High Recall Retrieval System/vectors/'
    
vocab = open('vocab/vocab_filtered.txt','r').read().splitlines()
word2idx = dict([(word,idx) for idx, word in enumerate(vocab)])
tokenizer = nlp.tokenizer

def _title_and_text(filename):
    tree = etree.parse(filename)
    root = tree.getroot()
    if root.find('.//HiddenText') is not None:
        text = (root.find('.//HiddenText').text)

    elif root.find('.//Text') is not None:
        text = (root.find('.//Text').text)

    else:
        text = None
    title = root.find('.//Title')
    if title is not None:
        title = title.text
    if not text is None:
        text = BeautifulSoup(text, parser='html.parser').get_text()
    return text,title
def get_bow_vector(filename):
    bow_vector = np.zeros(shape=(len(vocab)+1,))
    text,title = _title_and_text(filename)
    if not text is None and not title is None:
        for token in tokenizer(title+' '+text):
            t = token.text.replace('\n','')
            t = t.lower()
            if t in word2idx:
                bow_vector[word2idx[t]]+=1
            else:
                bow_vector[len(vocab)]+=1
    return bow_vector

filenames = []

for filename in os.listdir(TS_data_path):
    filenames.append(TS_data_path+filename)
for filename in os.listdir(GM_data_path):
    filenames.append(GM_data_path+filename)
filenames[-1]

'/home/ec2-user/SageMaker/data/The_Globe_and_Mail_with_DP_filter_by_article_type/1242359414.xml'

In [8]:
import pickle
import tqdm
for filename in tqdm.tqdm(filenames):
    bow_vector = get_bow_vector(filename)
    bow_vector_path = vectors_path+ filename.split('/')[-1][:-4] +  '.bow_vector'
    pickle.dump(bow_vector, open(bow_vector_path, 'wb'))

100%|██████████| 192433/192433 [31:28<00:00, 101.87it/s]


In [7]:
!rm vectors/id2vec.p


'/home/ec2-user/SageMaker/mariano/notebooks/03. High Recall Retrieval System/vectors/1242359414.bow_vector'